In [2]:
pip install pandas scikit-learn imbalanced-learn


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 13.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
data = pd.read_csv('/Users/gidaseul/Documents/GitHub/DS/다슬/Processed_data.csv')


In [22]:
fraud_counts = data['Is Fraud?'].value_counts()
fraud_counts

Is Fraud?
0    1642447
1       2321
Name: count, dtype: int64

In [27]:
#언더 샘플링
from sklearn.utils import resample

# 데이터프레임 복사 (원본 데이터 사용)
df = data.copy()

# 사기 거래 데이터와 비사기 거래 데이터 분리
fraud_data = df[df['Is Fraud?'] == 1]
non_fraud_data = df[df['Is Fraud?'] == 0]

# 언더샘플링 수행: 비사기 거래 데이터를 사기 거래 데이터 수와 맞춤
non_fraud_sample = resample(non_fraud_data, 
                            replace=False,  # 복원 추출하지 않음
                            n_samples=len(fraud_data),  # 사기 거래 데이터 수와 맞춤
                            random_state=42)

# 언더샘플링된 데이터와 사기 거래 데이터를 결합하여 학습 데이터 생성
under_sampled_data = pd.concat([fraud_data, non_fraud_sample])

# 특성과 타겟 데이터 분리
X_under = under_sampled_data.drop(columns=['Is Fraud?'])
y_under = under_sampled_data['Is Fraud?']


In [28]:
#오버샘플링
from imblearn.over_sampling import SMOTE

# 특성과 타겟 데이터 분리
X = df.drop(columns=['Is Fraud?'])
y = df['Is Fraud?']

# SMOTE 오버샘플링 적용
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)


ValueError: could not convert string to float: '2022-12-01'

In [29]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 언더샘플링된 데이터로 학습할 로지스틱 회귀 모델
model1 = LogisticRegression()
model1.fit(X_under, y_under)

# 오버샘플링된 데이터로 학습할 의사결정 나무 모델
model2 = DecisionTreeClassifier()
model2.fit(X_resampled, y_resampled)

# 앙상블 모델 생성 (Voting Classifier)
ensemble_model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')

# 학습 및 테스트 데이터 분할 (원본 데이터 사용)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 앙상블 모델 학습
ensemble_model.fit(X_train, y_train)

# 모델 평가
y_pred = ensemble_model.predict(X_test)
print("앙상블 모델의 성능:\n", classification_report(y_test, y_pred))


ValueError: could not convert string to float: '2023-07-01'